In [2]:
import pandas as pd
import numpy as np
import csv

In [3]:
data = pd.read_csv('train.csv')

In [4]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
data.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [6]:
data.shape

(7613, 5)

In [45]:
# Dependence of keyword on the target variable
data[['keyword', 'target']][(data.keyword.isna() != True) & (data.target == 1)].head()


,keyword,target
31,ablaze,1
33,ablaze,1
37,ablaze,1
38,ablaze,1
46,ablaze,1


In [46]:
data['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [47]:
data['location'].value_counts()

USA                            104
New York                        71
United States                   50
London                          45
Canada                          29
                              ... 
Eastlake, OH                     1
Hamilton, Ontario CA             1
paradise                         1
Stratford, CT                    1
Not where I want to be, yet      1
Name: location, Length: 3341, dtype: int64

In [48]:
# Dependence of location on target variable
data[['location', 'target']][(data.location.isna() != True) & (data.target == 0)].head()

,location,target
32,Est. September 2012 - Bristol,0
34,"Philadelphia, PA",0
35,"London, UK",0
36,Pretoria,0
39,Paranaque City,0


In [49]:
# dependence of id on target
print(data.id.head())
data.id.value_counts()

# Since we have unique id for every target hence this wont be a good feature

0    1
1    4
2    5
3    6
4    7
Name: id, dtype: int64


2047     1
9454     1
1234     1
3283     1
9430     1
        ..
581      1
6726     1
4679     1
10824    1
2049     1
Name: id, Length: 7613, dtype: int64

In [30]:
text = data.text

In [34]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

In [55]:
pst = PorterStemmer()
stp_words = stopwords.words('english')
regex = re.compile('[^a-zA-Z]')
new_text = []
for item in text:
    temp_li = []
    sentence = word_tokenize(item)
    for token in sentence:
        if token not in stp_words :
            token = pst.stem(token)
            if len(token) > 2:
                temp_li.append(regex.sub('', token))
    new_text.append(' '.join(temp_li))
        

In [56]:
len(new_text)

7613

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(lowercase = True, min_df = 2)
x = vectorizer.fit_transform(new_text)
features = vectorizer.get_feature_names()
x = pd.DataFrame(x.toarray(), columns = features)

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [80]:
y = data['target']
x_train, x_test, y_train, y_test = train_test_split(x, y,random_state = 42, stratify = y)


C:\Users\uru\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7773109243697479


array([[948, 138],
       [286, 532]], dtype=int64)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = 0)
model.fit(x_train, y_train)
print(accuracy_score(y_test, model.predict(x_test)))
confusion_matrix(y_test, model.predict(x_test))


In [83]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(random_state = 0)
model_xgb.fit(x_train, y_train)
print(accuracy_score(y_test, model_xgb.predict(x_test)))
confusion_matrix(y_test, model_xgb.predict(x_test))

0.756827731092437


array([[997,  89],
       [374, 444]], dtype=int64)